In [47]:
# example of a super learner model for binary classification
from numpy import hstack
from numpy import vstack
from numpy import asarray
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [48]:
#create a list of base models
def get_models():
  models = []
  models.append(SVC(kernel='rbf',probability=True,random_state=0))
  models.append(XGBClassifier(max_depth=9, eta = 0.4, gamma = 3, n_rounds = 100))
  return models

In [49]:
# collect out of fold predictions form k-fold cross validation
def get_out_of_fold_predictions(X, y, models):
	meta_X, meta_y = list(), list()
	# define split of data
	kfold = KFold(n_splits=10, shuffle=True)
	# enumerate splits
	for train_ix, test_ix in kfold.split(X):
		fold_yhats = list()
		# get data
		train_X, test_X = X[train_ix], X[test_ix]
		train_y, test_y = y[train_ix], y[test_ix]
		meta_y.extend(test_y)
		# fit and make predictions with each sub-model
		for model in models:
			model.fit(train_X, train_y)
			yhat = model.predict_proba(test_X)
			# store columns
			fold_yhats.append(yhat)
		# store fold yhats as columns
		meta_X.append(hstack(fold_yhats))
	return vstack(meta_X), asarray(meta_y)

In [50]:
# fit all base models on the training dataset
def fit_base_models(X, y, models):
	for model in models:
		model.fit(X, y)

In [51]:
# fit a meta model
def fit_meta_model(X, y):
	model = LogisticRegression(solver='liblinear')
	model.fit(X, y)
	return model

In [52]:
# make predictions with stacked model
def super_learner_predictions(X, models, meta_model):
	meta_X = list()
	for model in models:
		yhat = model.predict_proba(X)
		meta_X.append(yhat)
	meta_X = hstack(meta_X)
	# predict
	return meta_model.predict_proba(meta_X)

In [53]:
import io
import pandas as pd
df = pd.read_csv('X_trainData_column_modified_AMK.csv') #Enter the file name here.
# Dataset is now stored in a Pandas Dataframe

In [54]:
df

,DEL_CF_410280_d918T_307_iniB,DEL_CF_4408101_d102C_34_gid,DEL_F_4408101_d101C_gid_G34G,INS_CF_4242820_i2957G_986_embC,INS_F_409772_i410ATCT_iniB_G137G,INS_F_4247020_i506CC_embB_G169G,INS_F_4247970_i1456GT_embB_G486G,INS_N_4243642_i409GTCCCGGGGCGCCAC_embA_S137S,INS_P_3074519_G.117_thyA,INS_P_3074521_G.115_thyA,...,ahpC_NC_snp,ahpC_C_snp,gid_C_snp,iniA_C_snp,gyrB_F_indel,embC_C_snp,eis_NC_snp,rpsA_C_snp,rrl-rrs_NC_snp,AMK
0,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,1,0,1
1,0,0,0,0,0,0,0,0,1,0,...,0,1,1,1,0,0,1,1,0,1
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,0
3,0,0,0,0,0,0,0,1,1,0,...,0,1,0,1,0,0,1,1,0,0
4,0,0,0,0,0,0,0,0,1,0,...,1,0,0,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1357,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1358,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [55]:
X = df.iloc[:,:-1].values
X.shape

(1360, 219)

In [56]:
y = df.iloc[:,-1].values
y.shape

(1360,)

In [57]:
# get models
models = get_models()
# get out of fold predictions
meta_X, meta_y = get_out_of_fold_predictions(X, y, models)
print('Meta ', meta_X.shape, meta_y.shape)

[14:47:23] WARNING: ../src/learner.cc:516: 
Parameters: { n_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:47:25] WARNING: ../src/learner.cc:516: 
Parameters: { n_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:47:26] WARNING: ../src/learner.cc:516: 
Parameters: { n_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:47:28] WARNI

In [58]:
# fit base models
fit_base_models(X, y, models)

[14:47:40] WARNING: ../src/learner.cc:516: 
Parameters: { n_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [59]:
# fit the meta model
meta_model = fit_meta_model(meta_X, meta_y)

In [60]:
X_test = pd.read_csv('final_X_testData_column_modified_AMK.csv') #Enter the file name here.
# Dataset is now stored in a Pandas Dataframe

In [61]:
import numpy as np

In [62]:
X_test

,DEL_CF_410280_d918T_307_iniB,DEL_CF_4408101_d102C_34_gid,DEL_F_4408101_d101C_gid_G34G,INS_CF_4242820_i2957G_986_embC,INS_F_409772_i410ATCT_iniB_G137G,INS_F_4247020_i506CC_embB_G169G,INS_F_4247970_i1456GT_embB_G486G,INS_N_4243642_i409GTCCCGGGGCGCCAC_embA_S137S,INS_P_3074519_G.117_thyA,INS_P_3074521_G.115_thyA,...,ahpC_NC_snp,ahpC_C_snp,gid_C_snp,iniA_C_snp,gyrB_F_indel,embC_C_snp,eis_NC_snp,rpsA_C_snp,rrl-rrs_NC_snp,AMK
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
337,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?
338,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,?
339,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,?


In [63]:
X_val = X_test.iloc[:,:-1].values
X_val.shape

(341, 219)

In [64]:
yhat = super_learner_predictions(X_val, models, meta_model)


In [65]:
yhat

array([[0.02934735, 0.97065265],
       [0.03484316, 0.96515684],
       [0.02823871, 0.97176129],
       [0.02869383, 0.97130617],
       [0.02848188, 0.97151812],
       [0.02823871, 0.97176129],
       [0.0295153 , 0.9704847 ],
       [0.03102792, 0.96897208],
       [0.02836402, 0.97163598],
       [0.02975075, 0.97024925],
       [0.0288027 , 0.9711973 ],
       [0.029587  , 0.970413  ],
       [0.02860942, 0.97139058],
       [0.02961471, 0.97038529],
       [0.02977599, 0.97022401],
       [0.02955203, 0.97044797],
       [0.02835607, 0.97164393],
       [0.02866173, 0.97133827],
       [0.02858808, 0.97141192],
       [0.02817012, 0.97182988],
       [0.02828947, 0.97171053],
       [0.02869383, 0.97130617],
       [0.02893291, 0.97106709],
       [0.02764382, 0.97235618],
       [0.02813684, 0.97186316],
       [0.02779543, 0.97220457],
       [0.02857829, 0.97142171],
       [0.02773802, 0.97226198],
       [0.02838543, 0.97161457],
       [0.03042237, 0.96957763],
       [0.

In [66]:
submission_data = pd.read_csv('Y_testData_1_nolabels_AMK.csv')

In [67]:
ids=submission_data.iloc[:,:1].values

In [68]:
result=list()

for i in range(len(ids)):
    result.append([ ids[i,0], yhat[i,1]])

result.insert(0, ['ID','AMK'])
result

[['ID', 'AMK'],
 [1, 0.9706526505380411],
 [2, 0.9651568395911906],
 [3, 0.9717612893086013],
 [4, 0.9713061699041821],
 [5, 0.9715181161414297],
 [6, 0.9717612893086013],
 [7, 0.9704847046914301],
 [8, 0.9689720838074466],
 [9, 0.9716359753148921],
 [10, 0.9702492494899385],
 [11, 0.9711973044879609],
 [12, 0.9704130006536873],
 [13, 0.9713905828329186],
 [14, 0.9703852932917727],
 [15, 0.9702240117561881],
 [16, 0.9704479747113778],
 [17, 0.9716439306900212],
 [18, 0.9713382681671684],
 [19, 0.9714119181836571],
 [20, 0.9718298803940368],
 [21, 0.9717105331342683],
 [22, 0.9713061699041821],
 [23, 0.9710670905778671],
 [24, 0.9723561763694468],
 [25, 0.9718631553989561],
 [26, 0.9722045714081909],
 [27, 0.9714217058538839],
 [28, 0.9722619843470104],
 [29, 0.9716145712683774],
 [30, 0.9695776290585849],
 [31, 0.9706934029891704],
 [32, 0.9690072573362482],
 [33, 0.9704389896697588],
 [34, 0.9696470421090112],
 [35, 0.09897618046141156],
 [36, 0.9727066861239032],
 [37, 0.970484704691

In [69]:
output = pd.DataFrame(result)
output.to_csv('AMK_results.csv',header = False, index = False)